# Importing Packages

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from gensim import corpora, models
from pprint import pprint
import json
import pickle
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Declaring Variables

In [2]:
base_dir= "/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Datasets-v2/"
laptops = "laptops_joined.json"

# Defining Functions

In [3]:
def cleanItem(listX):
  newList=[]
  for item in listX:
    item = re.sub('[,./?!@]','',str(item))
    item = item.lower()  
    newList.append(item)
  return newList

In [4]:
def convertToWords(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(sentence, deacc=True, max_len=20))

In [5]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
 
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
 
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
 
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Data Preparation

In [ ]:
df_laptops=pd.read_json(base_dir+laptops)

In [ ]:
df_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60367 entries, 0 to 60366
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   overall      60367 non-null  int64 
 1   vote         21788 non-null  object
 2   verified     60367 non-null  bool  
 3   asin         60367 non-null  object
 4   reviewText   60363 non-null  object
 5   summary      60362 non-null  object
 6   category     60367 non-null  object
 7   description  60367 non-null  object
 8   title        60367 non-null  object
 9   rank         60367 non-null  object
 10  price        60367 non-null  object
dtypes: bool(1), int64(1), object(9)
memory usage: 5.1+ MB


In [ ]:
df_laptops['description'] = cleanItem(df_laptops['description'])
df_laptops['reviewText'] = cleanItem(df_laptops['reviewText'])

In [ ]:
df_laptops.asin.value_counts()

B009LL9VDG    1302
B00FNPD1VW     938
B00MMLV7VQ     598
B00NSHLUBU     584
B00L49X8E6     547
              ... 
B00CFGVEFM       5
B00DDFEXWA       5
B000YSY4RI       4
B0009PTD86       4
B000WGHY38       4
Name: asin, Length: 2584, dtype: int64

In [ ]:
df_laptops['rank'].head(100)

0     [>#59,248 in Computers &amp; Accessories (See ...
1     [>#59,248 in Computers &amp; Accessories (See ...
2     [>#59,248 in Computers &amp; Accessories (See ...
3     [>#59,248 in Computers &amp; Accessories (See ...
4     [>#59,248 in Computers &amp; Accessories (See ...
                            ...                        
95    [>#250,943 in Computers &amp; Accessories (See...
96    [>#250,943 in Computers &amp; Accessories (See...
97    [>#250,943 in Computers &amp; Accessories (See...
98    [>#250,943 in Computers &amp; Accessories (See...
99    [>#250,943 in Computers &amp; Accessories (See...
Name: rank, Length: 100, dtype: object

In [ ]:
df_laptops["rank_length"] = df_laptops["rank"].str.len()

In [ ]:
df_laptops['rank_length'].value_counts()

2     55532
0      2742
1      1052
4       715
3       282
35       44
Name: rank_length, dtype: int64

In [ ]:
rank1 = []
rank2 = []
for index, row in df_laptops.iterrows():
  if row['rank_length'] == 0:
    rank1.append('')
    rank2.append('')
  elif row['rank_length'] == 1:
    rank1.append(row['rank'][0])
    rank2.append('')
  else:
    rank1.append(row['rank'][0])
    rank2.append(row['rank'][1])

df_laptops['rank1'] = rank1
df_laptops['rank2'] = rank2

In [ ]:
df_laptops['rank1'].value_counts()

                                                      2742
>#4,464 in Computers & Accessories (See top 100)      1302
>#4,198 in Computers & Accessories (See top 100)       938
>#3,664 in Computers & Accessories (See top 100)       598
>#5,731 in Computers & Accessories (See top 100)       584
                                                      ... 
>#47,659 in Computers & Accessories (See top 100)        5
>#9,634 in Computers & Accessories (See top 100)         5
>#58,299 in Computers & Accessories (See top 100)        4
>#238,718 in Computers & Accessories (See top 100)       4
>#152,791 in Computers & Accessories (See top 100)       4
Name: rank1, Length: 2403, dtype: int64

In [ ]:
df_laptops['rank2'].value_counts()

                                                                       3794
>#719 in Computers & Accessories > Laptops > Traditional Laptops       1302
>#650 in Computers & Accessories > Laptops > Traditional Laptops        938
>#559 in Computers & Accessories > Laptops > Traditional Laptops        598
>#1,044 in Computers & Accessories > Laptops > Traditional Laptops      584
                                                                       ... 
>#59,955 in Computers & Accessories > Laptops > Traditional Laptops       5
>#2,146 in Computers & Accessories > Laptops > 2 in 1 Laptops             5
>#41,377 in Computers & Accessories > Laptops > Traditional Laptops       4
>#15,302 in Computers & Accessories > Laptops > Traditional Laptops       4
>#60,499 in Computers & Accessories > Laptops > Traditional Laptops       4
Name: rank2, Length: 2321, dtype: int64

In [ ]:
df_laptops['rank_1'] = df_laptops.rank1.str.replace('(>#,?)','')
df_laptops['rank_2'] = df_laptops.rank2.str.replace('(>#,?)','')

In [ ]:
df_laptops['rank_1'] = df_laptops.rank_1.str.replace(',','')
df_laptops['rank_2'] = df_laptops.rank_2.str.replace(',','')

In [ ]:
df_laptops['rank_1'] = df_laptops.rank_1.str.extract('(\d+)')
df_laptops['rank_2'] = df_laptops.rank_2.str.extract('(\d+)')

In [ ]:
df_laptops['rank_1'].value_counts()

4464      1302
4198       938
3664       598
5731       584
5917       547
          ... 
55643        5
47736        5
152791       4
238718       4
58299        4
Name: rank_1, Length: 2401, dtype: int64

In [ ]:
df_laptops.head()

In [ ]:
df_laptops = df_laptops.drop(columns=['rank1', 'rank2'])

In [ ]:
data_types_dict = {'vote': 'int32', 'price': 'float32', 'rank_1': 'int32', 'rank_2': 'int32'}

In [ ]:
values = {'vote': 0, 'price': 0, 'rank_1': 0, 'rank_2':0 }

In [ ]:
df_laptops['vote'] = df_laptops.vote.replace(',','', regex=True)

In [ ]:
df_laptops['price'] = df_laptops['price'].str.replace('$', '')

In [ ]:
df_laptops['price'] = df_laptops['price'].str.replace(',', '')

In [ ]:
df_laptops.loc[df_laptops['price'].str.contains('a'), 'price'] = '0'

In [ ]:
df_laptops['price'] = df_laptops['price'].str.replace(r'^\s*$', '0')

In [ ]:
df_laptops = df_laptops.fillna(value=values)

In [ ]:
df_laptops = df_laptops.astype(data_types_dict)

In [ ]:
df_laptops.to_json('/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Datasets-v2/laptops_cleaned.json')

## Dumping Pickle File

In [ ]:
with open(base_dir+'laptops_cleaned.pkl', 'wb') as f:
  pickle.dump(df_laptops, f)

In [ ]:
with open(base_dir+'laptops_cleaned.pkl', 'rb') as f:
  df_laptops = pickle.load(f)

# Descriptive Statistics

In [ ]:
len(pd.unique(df_laptops['asin']))

2584

In [ ]:
df_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60367 entries, 0 to 60366
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   overall      60367 non-null  int64  
 1   vote         60367 non-null  int32  
 2   verified     60367 non-null  int64  
 3   asin         60367 non-null  object 
 4   reviewText   60367 non-null  object 
 5   summary      60362 non-null  object 
 6   category     60367 non-null  object 
 7   description  60367 non-null  object 
 8   title        60367 non-null  object 
 9   rank         60367 non-null  object 
 10  price        60367 non-null  float32
 11  rank_length  60367 non-null  int64  
 12  rank_1       60367 non-null  int32  
 13  rank_2       60367 non-null  int32  
dtypes: float32(1), int32(3), int64(3), object(7)
memory usage: 6.0+ MB


In [ ]:
df_laptops.describe()

,overall,vote,verified,price,rank_length,rank_1,rank_2
count,60367.000000,60367.000000,60367.000000,60367.000000,60367.000000,6.036700e+04,60367.000000
mean,4.048785,5.353753,0.785694,84.396233,1.944142,2.776031e+04,6131.948681
std,1.330659,44.013008,0.410343,174.520325,1.020689,4.349496e+04,8161.963072
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,4.000000,0.000000,1.000000,0.000000,2.000000,7.906000e+03,960.000000
50%,5.000000,0.000000,1.000000,0.000000,2.000000,1.750600e+04,3601.000000
75%,5.000000,3.000000,1.000000,95.800003,2.000000,3.200500e+04,7394.000000
max,5.000000,6770.000000,1.000000,1628.880005,35.000000,1.335616e+06,95014.000000


#Data Cleaning, Lemmatization, LDA Model

## Laptops Review

### Bag of Words

In [ ]:
review_data = df_laptops.reviewText.astype('unicode').values.tolist()
review_words = list(convertToWords(review_data))
#with open(base_dir+'laundry_review_words.pkl','wb') as f:
#  pickle.dump(laundry_review_words, f)

In [ ]:
#with open(base_dir+'laundry_review_words.pkl','rb') as f:
#  laundry_review_words=pickle.load(f)

In [ ]:
bigram = gensim.models.Phrases(review_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[review_words], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['laptop', 'computer', 'pc', 'machine'])

In [ ]:
# Remove Stop Words
review_nostops = remove_stopwords(review_words)
 
# Form Bigrams
review_bigrams = make_bigrams(review_nostops)
 

 
print(review_bigrams[:1][0][:30])

['everything', 'least', 'bucks', 'bottom', 'line', 'ibook', 'exactly', 'extremely', 'portable', 'extremely', 'friendly', 'extremely', 'fast', 'compromise', 'screen', 'small', 'processor', 'slowest', 'available', 'mac', 'inches', 'screen', 'among', 'smallest', 'find', 'nowadays', 'super', 'sharp', 'fairly', 'pokey']


In [ ]:
import spacy
from __future__ import unicode_literals

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [ ]:
with open(base_dir+'laptop_review_bigrams.pkl','wb') as f:
  pickle.dump(review_bigrams, f)

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(review_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
 
print(data_lemmatized[:1][0][:30])

['least', 'buck', 'bottom', 'line', 'ibook', 'exactly', 'extremely', 'portable', 'extremely', 'friendly', 'extremely', 'fast', 'compromise', 'screen', 'small', 'processor', 'slow', 'available', 'inch', 'screen', 'small', 'find', 'nowadays', 'super', 'sharp', 'fairly', 'pokey', 'processor', 'still', 'adequate']


In [ ]:
with open(base_dir+'laptop_review_lemmatized.pkl', 'wb') as f:
  pickle.dump(data_lemmatized, f)

In [6]:
with open(base_dir+'laptop_review_lemmatized.pkl','rb') as f:
  data_lemmatized = pickle.load(f)  

### Corpus

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)   #dictionary find unique tokens/words, and assign unique ids for each of these words
 
# Create Corpus
texts = data_lemmatized
 
# Term Document Frequency
corpus_review = [id2word.doc2bow(text) for text in texts]
 
# View
print(corpus_review[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 2), (29, 2)]


In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)   #dictionary find unique tokens/words, and assign unique ids for each of these words

In [ ]:
with open(base_dir+'laptop_review_corpus.pkl','wb') as f:
  pickle.dump(corpus_review, f)

In [7]:
with open(base_dir+'laptop_review_corpus.pkl','rb') as f:
  corpus_review = pickle.load(f)

### LDA

In [27]:
# Build LDA model
lda_model_laptop_review = gensim.models.LdaMulticore(corpus=corpus_review,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [22]:
lda_model_laptop_review.save(base_dir+'lda_model_laptop_review.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [26]:
from pprint import pprint
 
# Print the Keyword in the 20 topics
pprint(lda_model_laptop_review.print_topics())
#doc_lda = lda_model_laundry_review[corpus_laundry_review]

[(0,
  '0.027*"get" + 0.019*"use" + 0.018*"buy" + 0.016*"go" + 0.016*"thing" + '
  '0.016*"really" + 0.013*"year" + 0.013*"would" + 0.013*"window" + '
  '0.012*"much"'),
 (1,
  '0.048*"great" + 0.040*"use" + 0.038*"work" + 0.033*"good" + 0.024*"love" + '
  '0.022*"fast" + 0.019*"price" + 0.019*"need" + 0.016*"buy" + '
  '0.014*"battery"'),
 (2,
  '0.040*"screen" + 0.026*"keyboard" + 0.016*"good" + 0.015*"feel" + '
  '0.015*"battery" + 0.013*"quality" + 0.013*"nice" + 0.012*"look" + '
  '0.012*"light" + 0.011*"display"'),
 (3,
  '0.022*"buy" + 0.016*"would" + 0.016*"work" + 0.015*"product" + '
  '0.015*"problem" + 0.014*"purchase" + 0.013*"return" + 0.013*"time" + '
  '0.012*"month" + 0.012*"back"'),
 (4,
  '0.036*"drive" + 0.029*"chromebook" + 0.026*"game" + 0.022*"hard" + '
  '0.019*"run" + 0.019*"memory" + 0.017*"ram" + 0.016*"play" + 0.013*"card" + '
  '0.013*"graphic"'),
 (5,
  '0.026*"use" + 0.018*"key" + 0.015*"get" + 0.015*"keyboard" + 0.015*"time" + '
  '0.013*"work" + 0.012*"s

In [16]:
from gensim.models import CoherenceModel
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_laptop_review, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4100957268415359


### Visualization

In [17]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=1f30d1cf642789169ee71e4f54ecef4f1cff474eff3a22caf924b7e1b4f62945
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [18]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [19]:
lda_model_laptop_review = gensim.models.ldamodel.LdaModel.load(base_dir+'lda_model_laptop_review.model')  

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_laptop_review, corpus_review, dictionary=lda_model_laptop_review.id2word)

In [21]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.110585  0.094367       1        1  17.650347
2     -0.134079 -0.170578       2        1  17.178305
5     -0.057085  0.128978       3        1  14.292384
1     -0.203040 -0.063576       4        1  11.370975
6      0.143838 -0.017023       5        1  10.619849
4      0.084882 -0.247829       6        1  10.196994
3     -0.031760  0.233673       7        1  10.000039
7      0.307830  0.041989       8        1   8.691107, topic_info=        Term          Freq         Total Category  logprob  loglift
178    great  27492.000000  27492.000000  Default  30.0000  30.0000
161      use  59655.000000  59655.000000  Default  29.0000  29.0000
100    drive  18072.000000  18072.000000  Default  28.0000  28.0000
59    screen  34326.000000  34326.000000  Default  27.0000  27.0000
165     work  34515.000000  34515.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
165     work   3616.254972  34515.968750   Topic8  -4.4449   0.1869
274  problem   2654.553293  15399.627208   Topic8  -4.7541   0.6848
29      find   2142.131516  12655.019172   Topic8  -4.9686   0.6666
53     power   2115.506239  11995.568258   Topic8  -4.9811   0.7076
72   upgrade   1868.585786   7022.817638   Topic8  -5.1052   1.1189

[573 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
205        1  0.120969     able
205        3  0.126053     able
205        4  0.088711     able
205        5  0.319253     able
205        6  0.171636     able
...      ...       ...      ...
167        6  0.011154     year
167        7  0.257681     year
17162      2  0.998899     yoga
1428       1  0.996306    young
16128      2  0.997779  zenbook

[1080 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 7, 5, 4, 8])

## Laptops Description

### Bag of Words

In [ ]:
des_data = df_laptops.description.astype('unicode').values.tolist()
des_words = list(convertToWords(des_data))
#with open(base_dir+'laundry_des_words.pkl','wb') as f:
#  pickle.dump(laundry_des_words, f)

In [ ]:
#import pickle
#with open(base_dir+'laundry_des_words.pkl','rb') as f:
#  laundry_des_words=pickle.load(f)

In [ ]:
bigram = gensim.models.Phrases(des_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[des_words], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['laptop', 'computer', 'pc'])

In [ ]:
import spacy
from __future__ import unicode_literals
 
# Remove Stop Words
des_nostops = remove_stopwords(des_words)
 
# Form Bigrams
des_bigrams = make_bigrams(des_nostops)
 
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
 
print(des_bigrams[:1][0][:30])
with open(base_dir+'laptop_des_bigrams.pkl','wb') as f:
  pickle.dump(des_bigrams, f)

['apple', 'ibook', 'gb', 'mb', 'cd', 'svga_enet']


In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
des_lemmatized = lemmatization(des_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
 
print(des_lemmatized[:1][0][:30])

['cd', 'svga_enet']


In [ ]:
with open(base_dir+'laptop_des_lemmatized.pkl','wb') as f:
  pickle.dump(des_lemmatized, f)

### Corpus

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(des_lemmatized)   #dictionary find unique tokens/words, and assign unique ids for each of these words
 
# Create Corpus
texts = des_lemmatized
 
# Term Document Frequency
corpus_des = [id2word.doc2bow(text) for text in texts]
 
# View
print(corpus_des[:1][0][:30])

[(0, 1), (1, 1)]


In [ ]:
with open(base_dir+'laptop_des_corpus.pkl','wb') as f:
  pickle.dump(corpus_des, f)

### LDA

In [ ]:
# Build LDA model
lda_model_laptop_des = gensim.models.LdaMulticore(corpus=corpus_des,
                                       id2word=id2word,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
lda_model_laptop_des.save(base_dir+'lda_model_laptop_des.model')

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:804: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [ ]:
from pprint import pprint
 
# Print the Keyword in the 10 topics
pprint(lda_model_laptop_des.print_topics())

[(0,
  '0.026*"video" + 0.022*"performance" + 0.017*"use" + 0.017*"get" + '
  '0.016*"photo" + 0.016*"design" + 0.014*"build" + 0.014*"view" + '
  '0.014*"speed" + 0.013*"device"'),
 (1,
  '0.031*"feature" + 0.026*"storage" + 0.022*"operate" + 0.017*"go" + '
  '0.016*"quality" + 0.015*"easy" + 0.015*"tablet" + 0.014*"space" + '
  '0.014*"provide" + 0.014*"perfect"'),
 (2,
  '0.074*"product" + 0.052*"strong" + 0.046*"renew" + 0.035*"supplier" + '
  '0.021*"provide" + 0.018*"pre" + 0.017*"office" + 0.017*"home" + 0.017*"box" '
  '+ 0.013*"wide"'),
 (3,
  '0.084*"battery" + 0.051*"warranty" + 0.051*"full" + 0.050*"day" + '
  '0.045*"inch" + 0.041*"display" + 0.040*"keyboard" + 0.037*"hour" + '
  '0.037*"touch" + 0.033*"screen"'),
 (4,
  '0.065*"window" + 0.054*"work" + 0.041*"test" + 0.036*"may" + 0.024*"look" + '
  '0.022*"back" + 0.020*"accessory" + 0.020*"hold" + 0.019*"come" + '
  '0.018*"include"'),
 (5,
  '0.135*"center" + 0.072*"div" + 0.068*"top" + 0.067*"height" + 0.067*"width" '

In [ ]:
from gensim.models import CoherenceModel
 
# Compute Coherence Score
coherence_model_lda_laptop_des = CoherenceModel(model=lda_model_laptop_des, texts=des_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda_laptop_des = coherence_model_lda_laptop_des.get_coherence()
print('Coherence Score: ', coherence_lda_laptop_des)

Coherence Score:  0.5796135346458424
